In [1]:
import sys
sys.path.append("/Users/grecco/Documents/code/jablonski/src")

import numpy as np
import pint
from jablonski import SpectroscopicSystem, transitions, util, initial
from jablonski import simulation
from poincare import Simulator

ureg = pint.get_application_registry()


In [2]:
from jablonski.states import SingletState, TripletState


class MySystem(SpectroscopicSystem):

    high: SingletState = initial(energy=3 * ureg.eV, default=1)
    low: SingletState = initial(energy=0 * ureg.eV, default=0)
    
    pump = transitions.Absorption(ground=low, excited=high, rate=1 / ureg.s)
    emission = transitions.Fluorescence(ground=low, excited=high, rate=1 / ureg.s)

In [3]:
from poincare import Simulator

sim = Simulator(
    MySystem, 
    # transform={"tmp": MySystem.emission.radiative_decay}
)
sol = sim.solve(save_at=np.arange(3) * ureg.s)
sol

/Users/grecco/Documents/code/poincare/src/poincare/simulator.py:169: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  solution = solver(problem, save_at=np.asarray(save_at))


,high,low
time,,
0,1.000000,0.000000
1,0.567657,0.432343
2,0.509163,0.490837


In [4]:
import pandas as pd
sol.index, sol.high.values

(Index([0, 1, 2], dtype='int64', name='time'),
 array([1.        , 0.56765698, 0.50916272]))

In [5]:
df = simulation.time_resolved_emission(
    MySystem, 
    simulation.pulse_excitation(MySystem.pump, 1, 1 * ureg.second),
    save_at=np.linspace(0, 10)# * ureg.s,
)

import matplotlib.pyplot as plt
print(df.attrs)
df.head()

{'lines': {'line0': Fluorescence(ground=low,excited=high,rate=1 / second)}}


,line0
time,
0.000000,1.0
0.204082,1.0
0.408163,0.8324492884917054
0.612245,0.721028445483796
0.816327,0.6469182303223009


In [9]:
df = simulation.steady_state_emission(
    MySystem, 
    MySystem.pump, 1,
)

import matplotlib.pyplot as plt
print(df.attrs)
df.head()

{'last_state': {high: np.float64(1.0), low: np.float64(0.0)}, 'lines': {'line0': Fluorescence(ground=low,excited=high,rate=1 / second)}}


,line0
time,
100,1.0
